In [65]:
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import matplotlib.image as mpimg
import seaborn as sns
import torch    
import cv2
import numpy as np
import requests
import io
import squarify
import yaml
import torchvision.transforms as transforms
import warnings
import google.generativeai as genai
import firebase_admin
import logging
import tempfile

from urllib.parse import urlparse
from firebase_admin import credentials, firestore, storage
from ultralytics import YOLO
from gradio_client import Client

random.seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
cred = credentials.Certificate("credential/all_credential.json")
firebase_admin.initialize_app(cred) if not firebase_admin._apps else firebase_admin.get_app()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [50]:
try:
    db = firestore.client()
    bucket = storage.bucket(name='planktosee-temp-file')
    bucket.make_public()
    logger.info("Connected to Firestore successfully.")
except Exception as e:
    logger.error(f"Error connecting to Firestore: {e}")

INFO:__main__:Connected to Firestore successfully.


In [ ]:
def extract_gcs_path(image_url):
    parsed_url = urlparse(image_url)
    path_parts = parsed_url.path.lstrip("/").split("/", 1)
    
    if len(path_parts) > 1:
        return path_parts[1] 
    return None  

def load_image_from_firebase(image_url):
    gcs_path = extract_gcs_path(image_url)
    if not gcs_path:
        raise ValueError(f"Invalid GCS URL: {image_url}")

    blob = bucket.blob(gcs_path)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        temp_file_path = temp_file.name 

    blob.download_to_filename(temp_file_path)
    
    return temp_file_path 

In [ ]:
import tempfile

def load_image_from_firebase(image_url):
    blob = bucket.blob(image_url)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        blob.download_to_filename(temp_file.name)
        temp_file_path = temp_file.name 

    return temp_file_path 

In [ ]:
def load_text_from_firebase(text_url):
    response = requests.get(text_url)
    
    if response.status_code == 200:
        text = response.text
    else:
        logger.error(f"Failed to load text from Firebase: {response.status_code}")
    return text

In [ ]:
def upload_image_to_firebase(image_path):
    blob = bucket.blob('images/' + os.path.basename(image_path))
    blob.upload_from_filename(image_path)
    blob.make_public()
    return blob.public_url

'https://storage.googleapis.com/planktosee-temp-file/images/signature.png'

In [ ]:
def remove_image_from_firebase(image_path):
    blob = bucket.blob('images/' + os.path.basename(image_path))
    blob.delete()
    logger.info(f"Image {image_path} removed from Firebase Storage.")

INFO:__main__:Image signature.png removed from Firebase Storage.


In [ ]:
def gemini(message):
    genai.configure(api_key="")

    chatbot = genai.GenerativeModel("gemini-1.5-flash-002")
    system = f"""
        Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
        1. *Hierarki Taksonomi dalam Plankton:*
        2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
        3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
        4. *Klasifikasi Berdasarkan Habitat:*  
        5. *Klasifikasi Berdasarkan Siklus Hidup:*  
        6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
        7. *Klasifikasi Berdasarkan Ukuran:*  
        """
    response = chatbot.generate_content(system)
    return response.text

def qwen2(message):
    client = Client("Qwen/Qwen2-57b-a14b-instruct-demo")
    result = client.predict(
        query=message,
        history=[],
        system=
            f"""
            Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
            1. *Hierarki Taksonomi dalam Plankton:*
            2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
            3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
            4. *Klasifikasi Berdasarkan Habitat:*  
            5. *Klasifikasi Berdasarkan Siklus Hidup:*  
            6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
            7. *Klasifikasi Berdasarkan Ukuran:*  
            """,
        api_name="/model_chat"
    )
    return result[1][0][1]

def deepseek(message):
    client = Client("Abubekersiraj/Deepseek")
    result = client.predict(
        message=message,
        system_message=f"""
            Saya ingin mendapatkan penjelasan mendalam tentang taksonomi dan klasifikasi plankton {message}. Berikut adalah beberapa aspek utama yang perlu dijelaskan:  
            1. *Hierarki Taksonomi dalam Plankton:*
            2. *Kelompok Utama Plankton Berdasarkan Taksonomi:*  
            3. *Klasifikasi Plankton Berdasarkan Kemampuan Bergerak:*  
            4. *Klasifikasi Berdasarkan Habitat:*  
            5. *Klasifikasi Berdasarkan Siklus Hidup:*  
            6. *Klasifikasi Berdasarkan Fungsi dalam Ekosistem:*  
            7. *Klasifikasi Berdasarkan Ukuran:*  
            """,
        max_tokens=2048,
        temperature=0.1,
        top_p=0.95,
        api_name="/chat"
    )
    return result

deepseek("Plankton")

Loaded as API: https://abubekersiraj-deepseek.hf.space ✔


INFO:httpx:HTTP Request: GET https://abubekersiraj-deepseek.hf.space/config "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://abubekersiraj-deepseek.hf.space/gradio_api/info?serialize=False "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://abubekersiraj-deepseek.hf.space/gradio_api/heartbeat/57eec3c5-fd98-4fb5-a5cd-017707d5dc48 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://abubekersiraj-deepseek.hf.space/gradio_api/queue/join "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://abubekersiraj-deepseek.hf.space/gradio_api/queue/data?session_hash=57eec3c5-fd98-4fb5-a5cd-017707d5dc48 "HTTP/1.1 200 OK"


'Plankton refers to a diverse group of organisms that float or drift passively in aquatic environments, including oceans, lakes, and rivers. These organisms are not capable of active swimming and are carried by water currents. Plankton play a crucial role in aquatic ecosystems as they form the base of many marine food webs and contribute significantly to the global carbon and nutrient cycles.\n\n1. Hierarchy of Taxonomy in Plankton:\n\nPlankton can be classified into different taxonomic groups based on their evolutionary relationships. The hierarchy of taxonomy for plankton includes:\n\n- Kingdom: Planktonic organisms belong to different kingdoms, including Protista, Plantae, Animalia, and Bacteria.\n- Phylum: Within each kingdom, planktonic organisms are classified into different phyla based on their morphological and physiological characteristics.\n- Class: Each phylum is further divided into classes based on shared characteristics.\n- Order: Orders are groups of related classes.\n- 

In [ ]:
def predict_img(model_option, llm_option, img_path):
    if model_option == "yolov8-detect":
        model = YOLO("../model/yolov8-detect.pt")
    elif model_option == "yolov8-acvit":
        model = YOLO("../model/yolov8-acvit.pt")
    else:
        return "Model tidak ditemukan"
    
    img = cv2.imread(img_path)
    img = cv2.resize(img, (864, 576))

    results = model(img)

    cv2.imwrite(
        filename="static/uploads/detect_img.jpg", 
        img=cv2.cvtColor(results[0].plot(), cv2.COLOR_BGR2RGB)
    )

    detected_classes = [model.names[int(box.cls)] for box in results[0].boxes]
    confidences = [float(box.conf) for box in results[0].boxes]

    if llm_option == "qwen":
        response = clean_text(qwen2(detected_classes))
    elif llm_option == "deepseek":
        response = clean_text(deepseek(detected_classes))
    else:
        response = "Pilih model LLM yang sesuai."

    return detected_classes, confidences, response

In [5]:
detected_classes, confidences, response = predict_img("yolov8-detect", "qwen", "noctilluca_scintillans.jpg")


0: 448x640 1 noctilluca_scintillans, 249.4ms
Speed: 12.1ms preprocess, 249.4ms inference, 8.2ms postprocess per image at shape (1, 3, 448, 640)
Loaded as API: https://qwen-qwen2-57b-a14b-instruct-demo.hf.space ✔
